<a href="https://colab.research.google.com/github/sanu0711/langChain-experiments/blob/main/Agents_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What Are Agents in LangChain?

An agent is an intelligent system that dynamically decides what actions to take based on user input and external tools. Instead of following a predefined flow, an agent can:

Reason about what action to take.

Choose the right tool (search, calculation, database lookup, etc.).

Execute the action and process results.

Iterate if needed (multi-step reasoning).

In [ ]:
!pip install -qU langchain-openai langchain

# Simple LangChain Agent

In [ ]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from google.colab import userdata

llm = ChatOpenAI(
    api_key = userdata.get('OPENAI_API_KEY'),
    model="gpt-4o",
    # temperature=0, # temperature controls the randomness of the output
    # max_tokens=None, # max_tokens controls the maximum number of tokens in the output
    # timeout=None, # timeout controls the maximum time to wait for the API to respond
    # max_retries=2,  # max_retries controls the maximum number of retries to attempt if the API fails
    #base_url="...", # base_url is the base URL for the OpenAI API
)


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

def square_number(n: str):
    """Squares a number"""
    return int(n) ** 2

math_tool = Tool(
    name="SquareTool",
    func=square_number,
    description="Calculates the square of a number. Input should be an integer."
)

agent = initialize_agent(
    tools=[math_tool],  # Adding our custom tool
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # This agent reasons before acting
    verbose=True  # Enables detailed logs
)

response = agent.run("What is the square of 4?")
print(response)



> Entering new AgentExecutor chain...
To find the square of 4, I need to calculate 4 times 4.

Action: SquareTool
Action Input: 4
Observation: 16
Thought:I now know the final answer.
Final Answer: The square of 4 is 16.

> Finished chain.
The square of 4 is 16.


In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import random

# Define the get_weather function as before
def get_weather(city: str) -> str:
    """Returns a fake weather report for a given city."""
    weather_conditions = ["Sunny", "Cloudy", "Rainy", "Stormy"]
    temperature = random.randint(15, 35)
    return f"The weather in {city} is {random.choice(weather_conditions)} with {temperature}°C."

# Create the get_weather tool using the Tool class directly
weather_tool = Tool(
    name="GetWeatherTool",
    func=get_weather,
    description="Gets the weather in a given city.",
    args_schema=None  # Removing args_schema for a simple tool
)

# Initialize the agent with both tools
agent = initialize_agent(
    tools=[math_tool, weather_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # This agent reasons before acting
    verbose=True
)

# Run the agent
response = agent.run("What is the square of 4?? Also, how is the weather in Mumbai?")
print(response)



> Entering new AgentExecutor chain...
I need to answer two questions: calculate the square of 4 and find out the current weather in Mumbai. I'll start with calculating the square of 4.

Action: SquareTool
Action Input: 4
Observation: 16
Thought:Now that I know the square of 4 is 16, I will find out the current weather in Mumbai.

Action: GetWeatherTool
Action Input: Mumbai
Observation: The weather in Mumbai is Rainy with 17°C.
Thought:I now know the final answers to both questions.

Final Answer: The square of 4 is 16. The weather in Mumbai is Rainy with 17°C.

> Finished chain.
The square of 4 is 16. The weather in Mumbai is Rainy with 17°C.


# Advanced LangChain Agents

Advanced agents can:

✅ Use multiple tools dynamically

✅ Handle complex reasoning

✅ Maintain memory of conversations

✅ Chain multiple steps intelligently

Example :
* Self-Ask Agent : `agent=AgentType.SELF_ASK_WITH_SEARCH`
* ReAct Agent: Reasoning + Acting :  `agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION`
* Structured Chat Agent
* Conversational Agent

## Creating a Custom Self-Ask Agent  (Uses only one tool at time)

In [ ]:
from langchain.tools import tool, Tool
from langchain.agents import initialize_agent, AgentType
import yfinance as yf

def clarify_stock_query(stock_symbol: str) -> str:
    """Asks the user to specify a stock symbol if missing."""
    known_stocks = ["AAPL", "TSLA", "BTC", "ETH"]
    if stock_symbol.upper() in known_stocks:
        return f"You selected {stock_symbol.upper()}."
    return "Please specify a valid stock symbol (e.g., AAPL, BTC, ETH)."

clarify_stock_query_tool = Tool(
    name="Intermediate Answer",
    func=clarify_stock_query,
    description="Asks the user to specify a stock symbol if missing."
)

# def get_stock_price(stock_symbol: str) -> str:
#     """Fetches the latest stock price from Yahoo Finance."""
#     try:
#         stock = yf.Ticker(stock_symbol)
#         price = stock.history(period="1d")["Close"].iloc[-1]
#         return f"The latest price of {stock_symbol.upper()} is ${price:.2f}."
#     except Exception:
#         return "Error retrieving stock price. Please try again."

# stock_price_tool = Tool(
#     name="Intermediate Answer",
#     func=get_stock_price,
#     description="Retrieves the latest stock price for a given stock symbol (e.g., AAPL, TSLA, BTC, ETH)."
# )


# Create Self-Ask Agent
agent = initialize_agent(
    tools=[clarify_stock_query_tool],
    llm=llm,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True
)

# Run the agent
response = agent.run(" TSLA?")
print(response)




> Entering new AgentExecutor chain...
Yes. TSLA typically refers to Tesla, Inc. on stock exchanges, so clarification is needed.

Follow up: Are you asking about the stock symbol TSLA?
Intermediate answer: Please specify a valid stock symbol (e.g., AAPL, BTC, ETH).
Yes, TSLA is the stock symbol for Tesla, Inc. on the stock exchange.

So the final answer is: Tesla, Inc. is represented by the stock symbol TSLA on the stock exchange.

> Finished chain.
Tesla, Inc. is represented by the stock symbol TSLA on the stock exchange.


## Agent Executor

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
import datetime
from langchain.agents import tool


@tool
def get_system_time(format: str = "%Y-%m-%d %H:%M:%S"):
    """ Returns the current date and time in the specified format """

    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime(format)
    return formatted_time


query = "What is the current time in London? (You are in India). Just show the current time and not the date"

prompt_template = hub.pull("hwchase17/react")

tools = [get_system_time]

agent = create_react_agent(llm, tools, prompt_template)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": query})



/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
To find out the current time in London when I am in India, I need to consider the time difference between the two locations. India is in the Indian Standard Time (IST) zone, which is UTC+5:30, whereas London is in the Greenwich Mean Time (GMT) or British Summer Time (BST) zone, depending on the time of the year. During BST, London is UTC+1, and otherwise, it is UTC+0. Given the difference, I will calculate the current time in London. First, I'll get the current time in India.

Action: get_system_time
Action Input: '%H:%M''09:56'I have the current time in India, which is 09:56. During BST (from the last Sunday in March to the last Sunday in October), London is 4 hours and 30 minutes behind India. Since today is sometime in October 2023, and BST is still in effect, I'll assume the time difference is 4 hours and 30 minutes. Thus, to convert the time to London time, I need to subtract 4 hours and 30 minutes from the current time in India.

09:56 (Ind

{'input': 'What is the current time in London? (You are in India). Just show the current time and not the date',
 'output': 'The current time in London is 05:26.'}

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain.schema.runnable import RunnableParallel
import datetime
import pytz
from langchain.agents import tool

# Initialize OpenAI model


@tool
def get_system_time(timezone: str = "UTC", format: str = "%Y-%m-%d %H:%M:%S"):
    """Returns the current date and time in the specified timezone and format."""
    try:
        tz = pytz.timezone(timezone)
    except pytz.UnknownTimeZoneError:
        return f"Unknown timezone: {timezone}"
    current_time = datetime.datetime.now(tz)
    return current_time.strftime(format)

@tool
def get_day_of_week(timezone: str = "UTC"):
    """Returns the current day of the week in the specified timezone."""
    try:
        tz = pytz.timezone(timezone)
    except pytz.UnknownTimeZoneError:
        return f"Unknown timezone: {timezone}"
    current_time = datetime.datetime.now(tz)
    return current_time.strftime("%A")

# Fetch the ReAct prompt template from LangChain Hub
prompt_template = hub.pull("hwchase17/react")

# Define tools
tools = [get_system_time, get_day_of_week]

# Create agents
agent_1 = create_react_agent(llm, [get_system_time], prompt_template)
agent_2 = create_react_agent(llm, [get_day_of_week], prompt_template)

# Create agent executors
agent_executor_1 = AgentExecutor(agent=agent_1, tools=[get_system_time], verbose=True)
agent_executor_2 = AgentExecutor(agent=agent_2, tools=[get_day_of_week], verbose=True)

# Execute both agents in parallel
parallel_agents = RunnableParallel(
    time_agent=lambda x: agent_executor_1.invoke({"input": "Get the current time in Europe/London."}),
    day_agent=lambda x: agent_executor_2.invoke({"input": "Get the current day in Europe/London."})
)

# Invoke both agents in parallel
parallel_agents.invoke({})



/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

> Entering new AgentExecutor chain...

I need to find out the current day in the Europe/London timezone.
Action: get_day_of_week
Action Input: "Europe/London"FridayI now know the final answer.
Final Answer: The current day in Europe/London is Friday.

> Finished chain.
To find the current time in Europe/London, I need to use the `get_system_time` tool with the appropriate timezone setting.

Action: get_system_time
Action Input: "Europe/London"2025-02-07 09:57:39I now know the final answer.

Final Answer: The current time in Europe/London is 2025-02-07 09:57:39.

> Finished chain.


{'time_agent': {'input': 'Get the current time in Europe/London.',
  'output': 'The current time in Europe/London is 2025-02-07 09:57:39.'},
 'day_agent': {'input': 'Get the current day in Europe/London.',
  'output': 'The current day in Europe/London is Friday.'}}

In [ ]:
prompt_template = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt_template

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')